In [1]:
import numpy as np 
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

1\. **Radioactive decay chain**

${\rm Tl}^{208}$ decays to ${\rm Pb}^{208}$ with a half-lieve of 3.052 minutes. Suppose to start with a sample of 1000 Thallium atoms and 0 of Lead atoms.

* Take steps in time of 1 second and at each time-step decide whether each Tl atom has decayed or not, accordingly to the probability $p(t)=1-2^{-t/\tau}$. Subtract the total number of Tl atoms that decayed at each step from the Tl sample and add them to the Lead one. Plot the evolution of the two sets as a function of time  
* Repeat the exercise by means of the inverse transform method: draw 1000 random numbers from the non-uniform probability distribution $p(t)=2^{-t/\tau}\frac{\ln 2}{\tau}$ to represent the times of decay of the 1000 Tl atoms. Make a plot showing the number of atoms that have not decayed as a function of time

2\. **Rutherford Scattering**

The scattering angle $\theta$ of $\alpha$ particles hitting a positively charged nucleus of a Gold atom ($Z=79$) follows the rule:

$$
\tan{\frac{1}{2} \theta} = \frac{Z e^2} {2\pi \epsilon_0 E b}
$$

where $E=7.7$ MeV and $b$ beam is the impact parameter. The beam is represented by a 2D gaussian distribution with $\sigma=a_0/100$ for both coordinates ($a_0$ being the Bohr radius). Assume 1 million $\alpha$ particles are shot on the gold atom.

Computing the fraction of particles that "bounce back",i.e. those particle whose scattering angle is greater than $\pi/2$ (which set a condition on the impact parameter $b$)

3\. **Monte Carlo integration: hit/miss vs mean value method**

Consider the function 

$$f(x) =\sin^2{\frac{1}{x(2-x)}}$$

* Compute the integral of $f(x)$ between 0 and 2 with the hit/miss method. Evaluate the error of your estimate
* Repeat the integral with the mean value method. Evaluate the error and compare it with the previous one

In [6]:
def f1(x):
    return np.sin(1/(x*(2-x)))**2

# To compute the integral of `func1(x)` between 0 and 2 using the hit/miss Monte Carlo method, we can follow these steps:

# 1. Define the number of random points `n` to be used for the estimation.
# 2. Generate `n` random points `x` in the interval [0, 2].
# 3. Generate `n` random points `y` in the interval [0, 1] (assuming the function `func1(x)` is bounded between 0 and 1).
# 4. Compute `func1(x)` for each randomly generated `x`.
# 5. Count the number of points `y` that are below `func1(x)`.
# 6. The integral is approximately equal to the count divided by `n`, times the area of the bounding box (2 in this case).
# 7. The error of the estimate can be computed as the square root of the variance divided by `n`.
    
n_steps = 1_000_000

def hitmiss(func, a, b, n):
    x = np.random.uniform(a, b, n)
    y = np.random.uniform(0, 1, n) # must br bounded 
    func_v = func(x)
    hits = np.sum(y < func_v)

    integral = hits/n * (b - a)
    error = (b - a) * np.sqrt((func_v**2).mean() - (func_v.mean())**2) / np.sqrt(n)
    
    return integral, error

integral_hit, error_hit = hitmiss(f1, 0, 2, n_steps)
print(f"Integral: {integral_hit}, Error: {error_hit}")


def meanint(func, a, b, n):
    x = np.random.uniform(a, b, n)
    integral = (b-a)/n * np.sum(func(x))
    error = (b - a) * np.sqrt((func(x)**2).mean() - (func(x).mean())**2) / np.sqrt(n)

    return integral, error

integral_mean, error_mean = meanint(f1, 0, 2, n_steps)
print(f"Integral: {integral_mean}, Error: {error_mean}")

Integral: 1.451586, Error: 0.0005275378857047181
Integral: 1.4508163857776484, Error: 0.0005276548632385677


4\. **Monte Carlo integration in high dimension**

* Start of by computing the area of a circle of unit radius, by integrating the function 

$$
f(x,y)=
\left\{
\begin{array}{ll}
      1 & x^2+y^2\le 1 \\
      0 & {\rm elsewhere}
\end{array} 
\right.
$$

* Generalize the result for a 10D sphere



In [11]:
def circle(x, y):
    if x**2 + y**2 <= 1:
        return 1
    else: return 0

n_steps = 1_000_000

points = np.random.uniform(-1, 1, (n_steps, 2))

count = 0
for i in range(n_steps):
    count += circle(points[i, 0], points[i, 1])

integral = 4*count/n_steps 

print(integral)

print('relative error:', np.abs(np.pi-integral)/np.pi)

def circleV(x):
    if np.sum([i**2 for i in x])<=1:
        return 1
    else:
        return 0
    
n_dim = 10

V_dim = 2**n_dim

points = np.random.uniform(-1, 1, (n_steps, n_dim))

count = 0
for i in range(n_steps):
    count += circleV(points[i, :])

integral = V_dim*count/n_steps 

print(integral)
print('relative error:', np.abs((np.pi**5/120)-integral)/(np.pi**5/120))


3.139436
relative error: 0.0006864841587049443
2.453504
relative error: 0.037903459685673394


5\. **Monte Carlo integration with importance sampling** 

Calculate the value of the integral:

$$
I=\int_0^1 \frac{x^{-1/2}}{e^x+1} dx
$$

using the importance sampling method with $w(x)=1/\sqrt{x}$. You should get a result about 0.84

In [12]:
def expfunc(x):
    return x**(-0.5) / (np.exp(x)+1)

def w(x):
    return 1/np.sqrt(x)

def importance(func, weight, a, b, n):

    def w_int(weight, a, b, n):
        x = np.random.uniform(a, b, n)
        int_w = (b-a)*np.sum(weight(x))/n
        return int_w
    
    x = np.random.uniform(a, b, n)
    integral = 1/n * np.sum(func(x)/weight(x)) * w_int(weight, a, b, n)

    return integral 

integral = importance(expfunc, w, 0, 1, 1_000_000)

print(integral)

0.7623057080838624
